# Walker-Penrose Killing tensor in Kerr spacetime

This Jupyter/SageMath notebook is relative to the lectures
[Geometry and physics of black holes](https://relativite.obspm.fr/blackholes/).

It focuses on the Killing tensor $K$ found by Walker & Penrose [[Commun. Math. Phys. **18**, 265 (1970)](https://doi.org/10.1007/BF01649445)]. 

The involved computations are based on tools developed through the [SageManifolds](https://sagemanifolds.obspm.fr) project.

*NB:* a version of SageMath at least equal to 8.2 is required to run this notebook:

In [1]:
version()

'SageMath version 9.1.rc1, Release Date: 2020-04-22'

First we set up the notebook to display mathematical objects using LaTeX rendering:

In [2]:
%display latex

To speed up the computations, we ask for running them in parallel on 8 threads:

In [3]:
Parallelism().set(nproc=8)

## Spacetime manifold

We declare the Kerr spacetime (or more precisely the Boyer-Lindquist domain of Kerr spacetime) as a 4-dimensional Lorentzian manifold:

In [4]:
M = Manifold(4, 'M', latex_name=r'\mathcal{M}', structure='Lorentzian')
print(M)

4-dimensional Lorentzian manifold M


Let us declare the **Boyer-Lindquist coordinates** via the method `chart()`, the argument of which is a string expressing the coordinates names, their ranges (the default is $(-\infty,+\infty)$) and their LaTeX symbols:

In [5]:
BL.<t,r,th,ph> = M.chart(r't r th:(0,pi):\theta ph:(0,2*pi):\phi') 
print(BL) ; BL

Chart (M, (t, r, th, ph))


Chart (M, (t, r, th, ph))

In [6]:
BL[0], BL[1]

(t, r)

In [7]:
BL.coord_range()

t: (-oo, +oo); r: (-oo, +oo); th: (0, pi); ph: (0, 2*pi)

<h2>Metric tensor</h2>

<p>The 2 parameters $m$ and $a$ of the Kerr spacetime are declared as symbolic variables:</p>

In [8]:
var('m, a', domain='real')

(m, a)

We get the (yet undefined) spacetime metric by 

In [9]:
g = M.metric()

<p>The metric is set by its components in the coordinate frame associated with Boyer-Lindquist coordinates, which is the current manifold's default frame:</p>

In [10]:
rho2 = r^2 + (a*cos(th))^2
Delta = r^2 -2*m*r + a^2
g[0,0] = -(1-2*m*r/rho2)
g[0,3] = -2*a*m*r*sin(th)^2/rho2
g[1,1], g[2,2] = rho2/Delta, rho2
g[3,3] = (r^2+a^2+2*m*r*(a*sin(th))^2/rho2)*sin(th)^2
g.display()

g = (2*m*r/(a^2*cos(th)^2 + r^2) - 1) dt*dt - 2*a*m*r*sin(th)^2/(a^2*cos(th)^2 + r^2) dt*dph + (a^2*cos(th)^2 + r^2)/(a^2 - 2*m*r + r^2) dr*dr + (a^2*cos(th)^2 + r^2) dth*dth - 2*a*m*r*sin(th)^2/(a^2*cos(th)^2 + r^2) dph*dt + (2*a^2*m*r*sin(th)^2/(a^2*cos(th)^2 + r^2) + a^2 + r^2)*sin(th)^2 dph*dph

<p>A matrix view of the components with respect to the manifold's default vector frame:</p>

In [11]:
g[:]

[                                  2*m*r/(a^2*cos(th)^2 + r^2) - 1                                                                 0                                                                 0                          -2*a*m*r*sin(th)^2/(a^2*cos(th)^2 + r^2)]
[                                                                0                         (a^2*cos(th)^2 + r^2)/(a^2 - 2*m*r + r^2)                                                                 0                                                                 0]
[                                                                0                                                                 0                                               a^2*cos(th)^2 + r^2                                                                 0]
[                         -2*a*m*r*sin(th)^2/(a^2*cos(th)^2 + r^2)                                                                 0                                                                 0 (2*a^2*m*r*sin(th)^2/(a^2*cos(th)^2 + r^2) + a^2 + r^2)*sin(th)^2]

<p>The list of the non-vanishing components:</p>

In [12]:
g.display_comp()

g_t,t = 2*m*r/(a^2*cos(th)^2 + r^2) - 1 
g_t,ph = -2*a*m*r*sin(th)^2/(a^2*cos(th)^2 + r^2) 
g_r,r = (a^2*cos(th)^2 + r^2)/(a^2 - 2*m*r + r^2) 
g_th,th = a^2*cos(th)^2 + r^2 
g_ph,t = -2*a*m*r*sin(th)^2/(a^2*cos(th)^2 + r^2) 
g_ph,ph = (2*a^2*m*r*sin(th)^2/(a^2*cos(th)^2 + r^2) + a^2 + r^2)*sin(th)^2

<h2>Levi-Civita Connection</h2>

<p>The Levi-Civita connection $\nabla$ associated with $g$:</p>

In [13]:
nabla = g.connection() ; print(nabla)

Levi-Civita connection nabla_g associated with the Lorentzian metric g on the 4-dimensional Lorentzian manifold M


<p>Let us verify that the covariant derivative of $g$ with respect to $\nabla$ vanishes identically:</p>

In [14]:
nabla(g).display()

nabla_g(g) = 0

<h2>Killing vectors</h2>
<p>The default vector frame on the spacetime manifold is the coordinate basis associated with Boyer-Lindquist coordinates:</p>

In [15]:
M.default_frame() is BL.frame()

True

In [16]:
BL.frame()

Coordinate frame (M, (d/dt,d/dr,d/dth,d/dph))

<p>Let us consider the first vector field of this frame:</p>

In [17]:
xi = BL.frame()[0] ; xi

Vector field d/dt on the 4-dimensional Lorentzian manifold M

In [18]:
print(xi)

Vector field d/dt on the 4-dimensional Lorentzian manifold M


<p>The 1-form associated to it by metric duality is</p>

In [19]:
xi_form = xi.down(g) ; xi_form.display()

-(a^2*cos(th)^2 - 2*m*r + r^2)/(a^2*cos(th)^2 + r^2) dt - 2*a*m*r*sin(th)^2/(a^2*cos(th)^2 + r^2) dph

<p>Its covariant derivative is</p>

In [20]:
nab_xi = nabla(xi_form) ; print(nab_xi) ; nab_xi.display()

Tensor field of type (0,2) on the 4-dimensional Lorentzian manifold M


(a^2*m*cos(th)^2 - m*r^2)/(a^4*cos(th)^4 + 2*a^2*r^2*cos(th)^2 + r^4) dt*dr + 2*a^2*m*r*cos(th)*sin(th)/(a^4*cos(th)^4 + 2*a^2*r^2*cos(th)^2 + r^4) dt*dth - (a^2*m*cos(th)^2 - m*r^2)/(a^4*cos(th)^4 + 2*a^2*r^2*cos(th)^2 + r^4) dr*dt + (a^3*m*cos(th)^2 - a*m*r^2)*sin(th)^2/(a^4*cos(th)^4 + 2*a^2*r^2*cos(th)^2 + r^4) dr*dph - 2*a^2*m*r*cos(th)*sin(th)/(a^4*cos(th)^4 + 2*a^2*r^2*cos(th)^2 + r^4) dth*dt + 2*(a^3*m*r + a*m*r^3)*cos(th)*sin(th)/(a^4*cos(th)^4 + 2*a^2*r^2*cos(th)^2 + r^4) dth*dph - (a^3*m*cos(th)^2 - a*m*r^2)*sin(th)^2/(a^4*cos(th)^4 + 2*a^2*r^2*cos(th)^2 + r^4) dph*dr - 2*(a^3*m*r + a*m*r^3)*cos(th)*sin(th)/(a^4*cos(th)^4 + 2*a^2*r^2*cos(th)^2 + r^4) dph*dth

<p>Let us check that the Killing equation is satisfied:</p>

In [21]:
nab_xi.symmetrize() == 0

True

<p>Similarly, let us check that $\frac{\partial}{\partial\phi}$ is a Killing vector:</p>

In [22]:
eta = BL.frame()[3] ; eta

Vector field d/dph on the 4-dimensional Lorentzian manifold M

In [23]:
nabla(eta.down(g)).symmetrize() == 0

True

## Principal null vectors

Let $k$ be the null vector tangent to the ingoing principal null geodesics associated
with their affine parameter $-r$; the expression of $k$ in term of the 3+1 ingoing Kerr coordinates $(\tilde{t}, r, \theta, \tilde\phi)$ is 
$$k = \frac{\partial}{\partial\tilde{t}} - \frac{\partial}{\partial\tilde{r}} $$
The expression of $k$ in terms of the Boyer-Lindquist coordinates is

In [24]:
k = M.vector_field(name='k')
k[:] = [(r^2 + a^2)/Delta, -1, 0, a/Delta]
k.display()

k = (a^2 + r^2)/(a^2 - 2*m*r + r^2) d/dt - d/dr + a/(a^2 - 2*m*r + r^2) d/dph

Regarding the null vector tangent to the outgoing principal null geodesics, we select
one associated with a (non-affine) parameter $\lambda$ that is regular accross the two Killing horizons:

In [25]:
el = M.vector_field(name='el', latex_name=r'\ell')
el[:] = [1/2, Delta/(2*(r^2+a^2)), 0, a/(2*(r^2+a^2))]
el.display()

el = 1/2 d/dt + 1/2*(a^2 - 2*m*r + r^2)/(a^2 + r^2) d/dr + 1/2*a/(a^2 + r^2) d/dph

Let us check that $k$ and $\ell$ are null vectors:

In [26]:
g(k,k).expr()

0

In [27]:
g(el,el).expr()

0

Their scalar product is $-\rho^2/(r^2 + a^2)$:

In [28]:
g(k,el).expr()

-(a^2*cos(th)^2 + r^2)/(a^2 + r^2)

Note that the scalar product (with respect to metric $g$) can also be computed by means of the method `dot`:

In [29]:
k.dot(el).expr()

-(a^2*cos(th)^2 + r^2)/(a^2 + r^2)

Let us check that $k$ is a geodesic vector, i.e. that it obeys  $\nabla_k k = 0$:

In [30]:
acc_k = nabla(k).contract(k)
acc_k.display()

0

We check that $\ell$ is a pregeodesic vector, i.e. that $\nabla_\ell \ell = \kappa_\ell \ell$ for some scalar field $\kappa_\ell$:

In [31]:
acc_l = nabla(el).contract(el)
acc_l.display()

-1/2*(a^2*m - m*r^2)/(a^4 + 2*a^2*r^2 + r^4) d/dt - 1/2*(a^4*m - 2*a^2*m^2*r + 2*m^2*r^3 - m*r^4)/(a^6 + 3*a^4*r^2 + 3*a^2*r^4 + r^6) d/dr - 1/2*(a^3*m - a*m*r^2)/(a^6 + 3*a^4*r^2 + 3*a^2*r^4 + r^6) d/dph

In [32]:
kappa_l = acc_l[[0]] / el[[0]]
kappa_l.display()

M --> R
(t, r, th, ph) |--> -(a^2*m - m*r^2)/(a^4 + 2*a^2*r^2 + r^4)

In [33]:
acc_l == kappa_l * el

True

## Walker-Penrose Killing tensor

We need the 1-forms associated to $k$ and $\ell$ by metric duality:

In [34]:
uk = k.down(g)
ul = el.down(g)

In [35]:
uk.display()

-dt - (a^2*cos(th)^2 + r^2)/(a^2 - 2*m*r + r^2) dr + (2*a^3*m*r*sin(th)^4 - (2*a^3*m*r - a^3*r^2 + 2*a*m*r^3 - a*r^4 - (a^5 + a^3*r^2)*cos(th)^2)*sin(th)^2)/(a^2*r^2 - 2*m*r^3 + r^4 + (a^4 - 2*a^2*m*r + a^2*r^2)*cos(th)^2) dph

In [36]:
uk[3]

(2*a^3*m*r*sin(th)^4 - (2*a^3*m*r - a^3*r^2 + 2*a*m*r^3 - a*r^4 - (a^5 + a^3*r^2)*cos(th)^2)*sin(th)^2)/(a^2*r^2 - 2*m*r^3 + r^4 + (a^4 - 2*a^2*m*r + a^2*r^2)*cos(th)^2)

In [37]:
uk[3].factor()

(a^4*cos(th)^2 + a^2*r^2*cos(th)^2 + 2*a^2*m*r*sin(th)^2 - 2*a^2*m*r + a^2*r^2 - 2*m*r^3 + r^4)*a*sin(th)^2/((a^2*cos(th)^2 + r^2)*(a^2 - 2*m*r + r^2))

In [38]:
uk[3] == a*sin(th)^2

True

In [39]:
uk[3] = a*sin(th)^2

In [40]:
uk.display()

-dt - (a^2*cos(th)^2 + r^2)/(a^2 - 2*m*r + r^2) dr + a*sin(th)^2 dph

In [41]:
ul.display()

-1/2*(a^2 - 2*m*r + r^2)/(a^2 + r^2) dt + 1/2*(a^2*cos(th)^2 + r^2)/(a^2 + r^2) dr + 1/2*(2*a^3*m*r*sin(th)^4 - (2*a^3*m*r - a^3*r^2 + 2*a*m*r^3 - a*r^4 - (a^5 + a^3*r^2)*cos(th)^2)*sin(th)^2)/(a^2*r^2 + r^4 + (a^4 + a^2*r^2)*cos(th)^2) dph

In [42]:
ul[3].factor()

1/2*(a^4*cos(th)^2 + a^2*r^2*cos(th)^2 + 2*a^2*m*r*sin(th)^2 - 2*a^2*m*r + a^2*r^2 - 2*m*r^3 + r^4)*a*sin(th)^2/((a^2*cos(th)^2 + r^2)*(a^2 + r^2))

In [43]:
ul[3] == a*sin(th)^2*(a^2 + r^2 - 2*m*r)/(2*(a^2 + r^2))

True

In [44]:
ul[3] = a*sin(th)^2*(a^2 + r^2 - 2*m*r)/(2*(a^2 + r^2))

In [45]:
ul.display()

-1/2*(a^2 - 2*m*r + r^2)/(a^2 + r^2) dt + 1/2*(a^2*cos(th)^2 + r^2)/(a^2 + r^2) dr + 1/2*(a^2 - 2*m*r + r^2)*a*sin(th)^2/(a^2 + r^2) dph

The Walker-Penrose Killing tensor $K$ is then formed as
$$ K = (r^2 + a^2) (\underline{\ell}\otimes \underline{k} + (\underline{k}\otimes \underline{\ell}) + r^2 g $$

In [46]:
K = 2*((r^2 + a^2)*(ul*uk)).symmetrize() + r^2*g
K.set_name('K')
print(K)

Field of symmetric bilinear forms K on the 4-dimensional Lorentzian manifold M


The non-vanishing components of $K$:

In [47]:
K.display_comp()

K_t,t = (a^2*r^2 + (a^4 - 2*a^2*m*r)*cos(th)^2)/(a^2*cos(th)^2 + r^2) 
K_t,ph = -(a^3*r^2 + a*r^4 + (a^5 - 2*a^3*m*r + a^3*r^2)*cos(th)^2)*sin(th)^2/(a^2*cos(th)^2 + r^2) 
K_r,r = -(a^4*cos(th)^4 + a^2*r^2*cos(th)^2)/(a^2 - 2*m*r + r^2) 
K_th,th = a^2*r^2*cos(th)^2 + r^4 
K_ph,t = -(a^3*r^2 + a*r^4 + (a^5 - 2*a^3*m*r + a^3*r^2)*cos(th)^2)*sin(th)^2/(a^2*cos(th)^2 + r^2) 
K_ph,ph = -((a^6 - 2*a^4*m*r + a^4*r^2)*sin(th)^6 - (a^6 - 2*a^4*m*r + a^4*r^2)*sin(th)^4 - (a^4*r^2 + 2*a^2*r^4 + r^6)*sin(th)^2)/(a^2*cos(th)^2 + r^2)

We may simplify a little bit some components:

In [48]:
K[0,3].factor()

-(a^4*cos(th)^2 - 2*a^2*m*r*cos(th)^2 + a^2*r^2*cos(th)^2 + a^2*r^2 + r^4)*a*sin(th)^2/(a^2*cos(th)^2 + r^2)

In [49]:
K[0,3] == - a*sin(th)^2*(r^2 + a^2 - 2*m*r*a^2*cos(th)^2/rho2)

True

In [50]:
K[0, 3] = - a*sin(th)^2*(r^2 + a^2 - 2*m*r*a^2*cos(th)^2/rho2)

In [51]:
K[1,1].factor()

-(a^2*cos(th)^2 + r^2)*a^2*cos(th)^2/(a^2 - 2*m*r + r^2)

In [52]:
K[3,3] == sin(th)^2 * (r^2*(r^2 + a^2) 
                       + a^2*sin(th)^2*(r^2 + a^2 - 2*m*r/rho2*a^2*cos(th)^2))

True

In [53]:
K[3,3] = sin(th)^2 * (r^2*(r^2 + a^2) 
                       + a^2*sin(th)^2*(r^2 + a^2 - 2*m*r/rho2*a^2*cos(th)^2))

In [54]:
K.display_comp()

K_t,t = (a^2*r^2 + (a^4 - 2*a^2*m*r)*cos(th)^2)/(a^2*cos(th)^2 + r^2) 
K_t,ph = (2*a^2*m*r*cos(th)^2/(a^2*cos(th)^2 + r^2) - a^2 - r^2)*a*sin(th)^2 
K_r,r = -(a^2*cos(th)^2 + r^2)*a^2*cos(th)^2/(a^2 - 2*m*r + r^2) 
K_th,th = a^2*r^2*cos(th)^2 + r^4 
K_ph,t = (2*a^2*m*r*cos(th)^2/(a^2*cos(th)^2 + r^2) - a^2 - r^2)*a*sin(th)^2 
K_ph,ph = -((2*a^2*m*r*cos(th)^2/(a^2*cos(th)^2 + r^2) - a^2 - r^2)*a^2*sin(th)^2 - (a^2 + r^2)*r^2)*sin(th)^2

In [55]:
DK = nabla(K)
print(DK)

Tensor field nabla_g(K) of type (0,3) on the 4-dimensional Lorentzian manifold M


In [56]:
DK.display_comp()

nabla_g(K)_t,r,ph = a*r*sin(th)^2 
nabla_g(K)_t,th,ph = (a^3 - 2*a*m*r + a*r^2)*cos(th)*sin(th) 
nabla_g(K)_t,ph,r = -a*r*sin(th)^2 
nabla_g(K)_t,ph,th = -(a^3 - 2*a*m*r + a*r^2)*cos(th)*sin(th) 
nabla_g(K)_r,t,ph = a*r*sin(th)^2 
nabla_g(K)_r,r,th = 2*(a^4*cos(th)^3 + a^2*r^2*cos(th))*sin(th)/(a^2 - 2*m*r + r^2) 
nabla_g(K)_r,th,r = -(a^4*cos(th)^3 + a^2*r^2*cos(th))*sin(th)/(a^2 - 2*m*r + r^2) 
nabla_g(K)_r,th,th = -a^2*r*cos(th)^2 - r^3 
nabla_g(K)_r,ph,ph = -(a^2*r + r^3)*sin(th)^2 
nabla_g(K)_th,t,ph = (a^3 - 2*a*m*r + a*r^2)*cos(th)*sin(th) 
nabla_g(K)_th,r,r = -(a^4*cos(th)^3 + a^2*r^2*cos(th))*sin(th)/(a^2 - 2*m*r + r^2) 
nabla_g(K)_th,r,th = -a^2*r*cos(th)^2 - r^3 
nabla_g(K)_th,th,r = 2*a^2*r*cos(th)^2 + 2*r^3 
nabla_g(K)_th,ph,ph = -(a^4 - 2*a^2*m*r + a^2*r^2)*cos(th)*sin(th)^3 
nabla_g(K)_ph,t,r = -a*r*sin(th)^2 
nabla_g(K)_ph,t,th = -(a^3 - 2*a*m*r + a*r^2)*cos(th)*sin(th) 
nabla_g(K)_ph,r,ph = -(a^2*r + r^3)*sin(th)^2 
nabla_g(K)_ph,th,ph = -(a^4 - 2*a^2*m*r + a^2*r^2)*cos(th)*sin(th)^3 
nabla_g(K)_ph,ph,r = 2*(a^2*r + r^3)*sin(th)^2 
nabla_g(K)_ph,ph,th = 2*(a^4 - 2*a^2*m*r + a^2*r^2)*cos(th)*sin(th)^3

Let us check that $K$ is a Killing tensor:

In [57]:
DK.symmetrize().display()

0

Equivalently, we may write, using index notation:

In [58]:
DK['_(abc)'].display()

0

In [59]:
pt = var('pt', latex_name=r'p^t')
pr = var('pr', latex_name=r'p^r')
pth = var('pth', latex_name=r'p^\theta')
pph = var('pph', latex_name=r'p^\phi')
p = M.vector_field(name='p')
p[:] = [pt, pr, pth, pph]
p.display()

p = pt d/dt + pr d/dr + pth d/dth + pph d/dph

In [60]:
K(p, p).expr()

-(a^6*pr^2*cos(th)^6 - a^2*pth^2*r^6 + 2*m*pth^2*r^7 - pth^2*r^8 - a^4*pt^2*r^2 + 2*a^2*m*pt^2*r^3 - a^2*pt^2*r^4 + (a^8*pph^2 - 4*a^6*m*pph^2*r - 4*a^4*m*pph^2*r^3 + a^4*pph^2*r^4 + 2*(a^6 + 2*a^4*m^2)*pph^2*r^2)*sin(th)^6 + (2*a^4*m*pth^2*r^3 - a^4*pth^2*r^4 - (a^6*pth^2 - 2*a^4*pr^2)*r^2)*cos(th)^4 - (a^8*pph^2 - 4*a^6*m*pph^2*r - 4*a^4*m*pph^2*r^3 + a^4*pph^2*r^4 + 2*(a^6 + 2*a^4*m^2)*pph^2*r^2)*sin(th)^4 + (4*a^2*m*pth^2*r^5 - 2*a^2*pth^2*r^6 - a^6*pt^2 + 4*a^4*m*pt^2*r + 2*a^2*m*pt^2*r^3 - (a^4 + 4*a^2*m^2)*pt^2*r^2 - (2*a^4*pth^2 - a^2*pr^2)*r^4)*cos(th)^2 + (2*m*pph^2*r^7 - pph^2*r^8 - (3*a^2*pph^2 - 2*a*pph*pt)*r^6 + 4*(a^2*m*pph^2 - a*m*pph*pt)*r^5 - (3*a^4*pph^2 - 4*a^3*pph*pt)*r^4 + 2*(a^4*m*pph^2 - 2*a^3*m*pph*pt)*r^3 - (a^6*pph^2 - 2*a^5*pph*pt)*r^2 + 2*(a^7*pph*pt - 4*a^5*m*pph*pt*r - 4*a^3*m*pph*pt*r^3 + a^3*pph*pt*r^4 + 2*(a^5 + 2*a^3*m^2)*pph*pt*r^2)*cos(th)^2)*sin(th)^2)/(a^2*r^2 - 2*m*r^3 + r^4 + (a^4 - 2*a^2*m*r + a^2*r^2)*cos(th)^2)

In [61]:
s = K(p, p).expr()
s

-(a^6*pr^2*cos(th)^6 - a^2*pth^2*r^6 + 2*m*pth^2*r^7 - pth^2*r^8 - a^4*pt^2*r^2 + 2*a^2*m*pt^2*r^3 - a^2*pt^2*r^4 + (a^8*pph^2 - 4*a^6*m*pph^2*r - 4*a^4*m*pph^2*r^3 + a^4*pph^2*r^4 + 2*(a^6 + 2*a^4*m^2)*pph^2*r^2)*sin(th)^6 + (2*a^4*m*pth^2*r^3 - a^4*pth^2*r^4 - (a^6*pth^2 - 2*a^4*pr^2)*r^2)*cos(th)^4 - (a^8*pph^2 - 4*a^6*m*pph^2*r - 4*a^4*m*pph^2*r^3 + a^4*pph^2*r^4 + 2*(a^6 + 2*a^4*m^2)*pph^2*r^2)*sin(th)^4 + (4*a^2*m*pth^2*r^5 - 2*a^2*pth^2*r^6 - a^6*pt^2 + 4*a^4*m*pt^2*r + 2*a^2*m*pt^2*r^3 - (a^4 + 4*a^2*m^2)*pt^2*r^2 - (2*a^4*pth^2 - a^2*pr^2)*r^4)*cos(th)^2 + (2*m*pph^2*r^7 - pph^2*r^8 - (3*a^2*pph^2 - 2*a*pph*pt)*r^6 + 4*(a^2*m*pph^2 - a*m*pph*pt)*r^5 - (3*a^4*pph^2 - 4*a^3*pph*pt)*r^4 + 2*(a^4*m*pph^2 - 2*a^3*m*pph*pt)*r^3 - (a^6*pph^2 - 2*a^5*pph*pt)*r^2 + 2*(a^7*pph*pt - 4*a^5*m*pph*pt*r - 4*a^3*m*pph*pt*r^3 + a^3*pph*pt*r^4 + 2*(a^5 + 2*a^3*m^2)*pph*pt*r^2)*cos(th)^2)*sin(th)^2)/(a^2*r^2 - 2*m*r^3 + r^4 + (a^4 - 2*a^2*m*r + a^2*r^2)*cos(th)^2)

### Metric dual of the Killing tensor ($K^{ab}$)

In [62]:
uK = K.up(g)
uK.set_name('uK', latex_name=r'\hat{K}')
print(uK)

Tensor field uK of type (2,0) on the 4-dimensional Lorentzian manifold M


In [63]:
uK.display_comp()

uK^t,t = (a^6 + 2*a^4*r^2 + a^2*r^4 - (a^6 + a^4*r^2 + 2*a^2*m*r^3)*sin(th)^2)/(a^2*r^2 - 2*m*r^3 + r^4 + (a^4 - 2*a^2*m*r + a^2*r^2)*cos(th)^2) 
uK^t,ph = (a^3*r^2 - 2*a*m*r^3 + a*r^4 + (a^5 + a^3*r^2)*cos(th)^2)/(a^2*r^2 - 2*m*r^3 + r^4 + (a^4 - 2*a^2*m*r + a^2*r^2)*cos(th)^2) 
uK^r,r = -(a^4 - 2*a^2*m*r + a^2*r^2)*cos(th)^2/(a^2*cos(th)^2 + r^2) 
uK^th,th = r^2/(a^2*cos(th)^2 + r^2) 
uK^ph,t = (a^3*r^2 - 2*a*m*r^3 + a*r^4 + (a^5 + a^3*r^2)*cos(th)^2)/(a^2*r^2 - 2*m*r^3 + r^4 + (a^4 - 2*a^2*m*r + a^2*r^2)*cos(th)^2) 
uK^ph,ph = (a^4*sin(th)^4 - a^4*sin(th)^2 - a^2*r^2 + 2*m*r^3 - r^4)/((a^4 - 2*a^2*m*r + a^2*r^2)*cos(th)^4 - a^2*r^2 + 2*m*r^3 - r^4 - (a^4 - 2*a^2*m*r + 2*m*r^3 - r^4)*cos(th)^2)

In [64]:
uK[0,0] == a^2/Delta*(r^2 + a^2 - 2*m*r^3*sin(th)^2/rho2)

True

In [65]:
uK[0,0] = a^2/Delta*(r^2 + a^2 - 2*m*r^3*sin(th)^2/rho2)

In [66]:
uK[0,3] == a/Delta*(r^2 + a^2 - 2*m*r^3/rho2)

True

In [67]:
uK[0,3] = a/Delta*(r^2 + a^2 - 2*m*r^3/rho2)

In [68]:
uK[1,1] == - Delta*a^2*cos(th)^2/rho2

True

In [69]:
uK[1,1] =  - Delta*a^2*cos(th)^2/rho2

In [70]:
uK[3,3] == (a^2 + r^2/sin(th)^2*(1 - 2*m*r/rho2))/Delta

True

In [71]:
uK[3,3] = (a^2 + r^2/sin(th)^2*(1 - 2*m*r/rho2))/Delta

In [72]:
uK.display_comp()

uK^t,t = -(2*m*r^3*sin(th)^2/(a^2*cos(th)^2 + r^2) - a^2 - r^2)*a^2/(a^2 - 2*m*r + r^2) 
uK^t,ph = -(2*m*r^3/(a^2*cos(th)^2 + r^2) - a^2 - r^2)*a/(a^2 - 2*m*r + r^2) 
uK^r,r = -(a^2 - 2*m*r + r^2)*a^2*cos(th)^2/(a^2*cos(th)^2 + r^2) 
uK^th,th = r^2/(a^2*cos(th)^2 + r^2) 
uK^ph,t = (a^3*r^2 - 2*a*m*r^3 + a*r^4 + (a^5 + a^3*r^2)*cos(th)^2)/(a^2*r^2 - 2*m*r^3 + r^4 + (a^4 - 2*a^2*m*r + a^2*r^2)*cos(th)^2) 
uK^ph,ph = (a^2 - (2*m*r/(a^2*cos(th)^2 + r^2) - 1)*r^2/sin(th)^2)/(a^2 - 2*m*r + r^2)

### Killing tensor leading directly to Carter constant Q

We introduce first the Killing vector $V := \eta + a \xi$:

In [73]:
V = eta + a*xi
V.display()

a d/dt + d/dph

In [74]:
V_form = V.down(g)
V_form.display()

-(a*r^2 + (a^3 - 2*a*m*r)*cos(th)^2)/(a^2*cos(th)^2 + r^2) dt + (2*a^2*m*r*sin(th)^4 - (2*a^2*m*r - a^2*r^2 - r^4 - (a^4 + a^2*r^2)*cos(th)^2)*sin(th)^2)/(a^2*cos(th)^2 + r^2) dph

and form the Killing tensor 
$$\tilde{K} = K - \underline{V}\otimes\underline{V}$$

In [75]:
K2 = K - V_form*V_form
K2.set_name('tK', latex_name=r'\tilde{K}')
K2.display_comp()

tK_t,t = 2*(a^2*m*r^3*cos(th)^2 + (a^4*m*r - 2*a^2*m^2*r^2)*cos(th)^4)/(a^4*cos(th)^4 + 2*a^2*r^2*cos(th)^2 + r^4) 
tK_t,ph = 2*((a^5*m*r - 2*a^3*m^2*r^2 + a^3*m*r^3)*cos(th)^6 - (a^5*m*r - 2*a^3*m^2*r^2 - a*m*r^5)*cos(th)^4 - (a^3*m*r^3 + a*m*r^5)*cos(th)^2)/(a^4*cos(th)^4 + 2*a^2*r^2*cos(th)^2 + r^4) 
tK_r,r = -(a^2*cos(th)^2 + r^2)*a^2*cos(th)^2/(a^2 - 2*m*r + r^2) 
tK_th,th = a^2*r^2*cos(th)^2 + r^4 
tK_ph,t = 2*((a^5*m*r - 2*a^3*m^2*r^2 + a^3*m*r^3)*cos(th)^6 - (a^5*m*r - 2*a^3*m^2*r^2 - a*m*r^5)*cos(th)^4 - (a^3*m*r^3 + a*m*r^5)*cos(th)^2)/(a^4*cos(th)^4 + 2*a^2*r^2*cos(th)^2 + r^4) 
tK_ph,ph = ((2*a^6*m*r + 4*a^4*m*r^3 - a^4*r^4 - (a^6 + 4*a^4*m^2)*r^2)*cos(th)^8 - (4*a^6*m*r + 6*a^4*m*r^3 + a^4*r^4 - 4*a^2*m*r^5 + 2*a^2*r^6 - (a^6 + 8*a^4*m^2)*r^2)*cos(th)^6 + (2*a^6*m*r - 4*a^4*m^2*r^2 + 2*a^4*r^4 - 8*a^2*m*r^5 + a^2*r^6 - r^8)*cos(th)^4 + (2*a^4*m*r^3 + 4*a^2*m*r^5 + a^2*r^6 + r^8)*cos(th)^2)/(a^4*cos(th)^4 + 2*a^2*r^2*cos(th)^2 + r^4)

Let us check that $\tilde{K}$ is indeed a Killing tensor:

In [76]:
nabla(K2).symmetrize().display()

0

We check that
$$ \tilde{K}^{\alpha\beta} = - a^2 \cos^2 \theta \, g^{\alpha\beta} 
    + \mathrm{diag}(-a^2\cos^2\theta, 0, 1, \tan^{-2}\theta)^{\alpha\beta}$$ 

In [77]:
T = K2 + (a^2*cos(th)^2)*g
uT = T.up(g)
uT[:]

[     -a^2*cos(th)^2                   0                   0                   0]
[                  0                   0                   0                   0]
[                  0                   0                   1                   0]
[                  0                   0                   0 cos(th)^2/sin(th)^2]